In [1]:
import os

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import torchvision
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch.nn.functional as F

In [4]:
xy_data_Path = "C:\\Users\\Matth\\OneDrive\\Desktop\\Summer\\Python\\Quadratic_Guesser\\.gitignore\\xy_data.csv"
xy_data = pd.read_csv(xy_data_Path)

In [5]:
abc_data_Path = "C:\\Users\\Matth\\OneDrive\\Desktop\\Summer\\Python\\Quadratic_Guesser\\.gitignore\\abc_data.csv"
abc_data = pd.read_csv(abc_data_Path)

In [6]:
all_data=pd.DataFrame.join(xy_data,abc_data)

In [7]:
training_data, validation_data = train_test_split(all_data, test_size=0.2, random_state=42)

In [8]:
X_traine = training_data.iloc[0:8000,0:500]
y_traine = training_data.iloc[0:8000,500:503]

X_vale = validation_data.iloc[0:2000:,0:500]
y_vale = validation_data.iloc[0:2000,500:503]

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_traine)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_traine.to_numpy(), dtype=torch.float32)

X_val_scaled = scaler.transform(X_vale)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val = torch.tensor(y_vale.to_numpy(), dtype=torch.float32)

In [10]:
dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

valset = TensorDataset(X_val, y_val)
val_loader = DataLoader(valset, batch_size=64, shuffle=True)

In [11]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out



In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 500  # Number of y-coordinates
hidden_size = 128
output_size = 3   # Number of coefficients
model = FeedforwardNN(input_size, hidden_size, output_size).to(device)


criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005, weight_decay = 1e-5)

In [13]:
# Initialize weights
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)
model.apply(weights_init)

FeedforwardNN(
  (fc1): Linear(in_features=500, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=3, bias=True)
)

In [14]:
def validate_model(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

    return val_loss / len(val_loader.dataset)

In [15]:
num_epochs =   100
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
    
    val_loss = validate_model(model, val_loader, criterion, device)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}; Val loss = {val_loss}')

Epoch [1/100], Loss: 194.8517; Val loss = 1966.5705599060059
Epoch [2/100], Loss: 238.3549; Val loss = 1927.1042683105468
Epoch [3/100], Loss: 108.0880; Val loss = 1885.4487338867189
Epoch [4/100], Loss: 188.7159; Val loss = 1833.0190469360352
Epoch [5/100], Loss: 92.4311; Val loss = 1781.734815673828
Epoch [6/100], Loss: 65.5716; Val loss = 1716.6899785766602
Epoch [7/100], Loss: 14748.5225; Val loss = 1647.6000290527343
Epoch [8/100], Loss: 85.6491; Val loss = 1573.9204282226563
Epoch [9/100], Loss: 71.1064; Val loss = 1489.3744179077148
Epoch [10/100], Loss: 65.2948; Val loss = 1429.4293624267577
Epoch [11/100], Loss: 47.4144; Val loss = 1347.861667175293
Epoch [12/100], Loss: 79.1295; Val loss = 1265.4177485046387
Epoch [13/100], Loss: 967.1780; Val loss = 1179.0775624084472
Epoch [14/100], Loss: 47.3053; Val loss = 1101.9442671203612
Epoch [15/100], Loss: 171.6196; Val loss = 1014.0583553771972
Epoch [16/100], Loss: 35.4413; Val loss = 918.6614729309082
Epoch [17/100], Loss: 88861

In [16]:
import random as rand
def XandYQuadValues(x1,x2,c):
# Calculate 'a' from the y-intercept and x-intercepts
    if x1 * x2 != 0:
        a = c / (x1 * x2)
    else:
        a = 1  # Default to 1 if x1 or x2 is zero to avoid division by zero

    # Define the range of x values to cover the extended plot
    x_min = -50 # Extending beyond the smallest intercept
    x_max = 50  # Extending beyond the largest intercept
    x = np.linspace(x_min, x_max, 500)

    # Calculate the corresponding y values based on the quadratic formula
    y = a * (x - x1) * (x - x2)

    return x,y,a

In [17]:
xydf = pd.DataFrame()
abcdf = pd.DataFrame()
x = pd.DataFrame([list([1,2,3])])

for i in range(0,1):
    x_int1 = rand.uniform(-25,25)
    x_int2 = rand.uniform(-25,25)
    factor = rand.uniform(-25,25)
    x,y,a = XandYQuadValues(x_int1,x_int2,factor)
    xdf = pd.DataFrame([list(x)])
    ydf = pd.DataFrame([list(y)])
    b = a*-(x_int1+x_int2)
    c = a*x_int1*x_int2
    if i == 0:
        xydf = pd.concat([xdf,ydf])
        abcdf = pd.DataFrame([[a, b, c]])
    else:
        xydf = pd.concat([xydf,ydf])
        abcdf = pd.concat([abcdf, pd.DataFrame([list([a, b, c])])])


xydfnp = xydf.iloc[1,:].to_numpy()
xydfnp = np.expand_dims(xydfnp, axis=1)
print(xydfnp.shape)
test_tensor = torch.tensor(xydfnp, dtype=torch.float32)
print(abcdf)

(500, 1)
          0         1         2
0 -0.047473  1.278354 -7.046773


In [18]:

model.eval()

with torch.no_grad():  # Turn off gradients since we're in inference mode
    
    test_tensor = test_tensor.view(1, -1)
    test_tensor_scaled = scaler.transform(test_tensor)
    outputs = model(torch.tensor(test_tensor_scaled, dtype=torch.float32))



predicted_labels = outputs.detach().numpy()  # Convert to numpy 
print("Predicted Output:", predicted_labels)

Predicted Output: [[-0.04483857  1.2781416  -7.0726333 ]]


c:\Users\Matth\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
